# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921717953466                   -0.69    5.8         
  2   -7.926170381825       -2.35       -1.22    1.0    212ms
  3   -7.926837166086       -3.18       -2.37    1.9    234ms
  4   -7.926861501315       -4.61       -3.02    2.6    251ms
  5   -7.926861641136       -6.85       -3.37    2.2    262ms
  6   -7.926861668858       -7.56       -3.77    1.8    215ms
  7   -7.926861680378       -7.94       -4.32    1.1    197ms
  8   -7.926861681771       -8.86       -4.90    1.9    218ms
  9   -7.926861681858      -10.06       -5.20    2.1    269ms
 10   -7.926861681871      -10.89       -5.78    1.2    203ms
 11   -7.926861681873      -11.75       -6.57    1.9    220ms
 12   -7.926861681873      -13.07       -7.49    2.2    245ms
 13   -7.926861681873   +  -14.75       -7.91    2.8    260ms
 14   -7.926861681873      -15.05       -8.44    2.5    235ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921694524741                   -0.69    5.6         
  2   -7.926165938123       -2.35       -1.22    1.0    211ms
  3   -7.926836419178       -3.17       -2.37    1.6    232ms
  4   -7.926861515312       -4.60       -3.00    3.0    257ms
  5   -7.926861628760       -6.95       -3.31    2.1    288ms
  6   -7.926861666055       -7.43       -3.71    1.2    203ms
  7   -7.926861680673       -7.84       -4.43    1.2    202ms
  8   -7.926861681828       -8.94       -5.03    2.4    235ms
  9   -7.926861681852      -10.62       -5.12    1.9    267ms
 10   -7.926861681872      -10.70       -5.96    1.0    196ms
 11   -7.926861681873      -11.98       -6.94    2.1    233ms
 12   -7.926861681873      -13.69       -7.08    3.1    259ms
 13   -7.926861681873   +    -Inf       -7.92    1.0    216ms
 14   -7.926861681873      -14.45       -8.85    3.2    248ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921683203477                   -0.69    5.8         
  2   -7.926171238295       -2.35       -1.22    1.0    227ms
  3   -7.926839954462       -3.17       -2.37    1.6    250ms
  4   -7.926864888225       -4.60       -2.99    2.9    282ms
  5   -7.926865027755       -6.86       -3.27    1.9    278ms
  6   -7.926865075634       -7.32       -3.69    1.2    205ms
  7   -7.926865091867       -7.79       -4.44    1.6    219ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       
